In [8]:
# API to fetch the dataset from kaggle
# !kaggle datasets download -d kazanova/sentiment140

In [9]:
# import zipfile
# import os
# # Unzipping file
# with zipfile.ZipFile('sentiment140.zip', 'r') as zip_ref:
#   zip_ref.extractall()

In [10]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [11]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jason\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
# printing stopwords in English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Processing

In [13]:
# Loading data from csv file to pandas dataframe
df = pd.read_csv('data/data.csv', encoding='ISO-8859-1')

In [14]:
# Checking # of rows/cols
df.shape

(1599999, 6)

In [15]:
df.head(5)

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [16]:
column_names =['target', 'id', 'date', 'flag', 'user', 'text']
df = pd.read_csv('data/data.csv', encoding='ISO-8859-1', names=column_names)

In [17]:
df

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [18]:
# Check for missing values
df.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [19]:

df.describe()

,target,id
count,1.600000e+06,1.600000e+06
mean,2.000000e+00,1.998818e+09
std,2.000001e+00,1.935761e+08
min,0.000000e+00,1.467810e+09
25%,0.000000e+00,1.956916e+09
50%,2.000000e+00,2.002102e+09
75%,4.000000e+00,2.177059e+09
max,4.000000e+00,2.329206e+09


In [20]:
# Checking distributions of target column
df['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

- Equal distribution of target column which will be great for checking accuracy metrics

In [21]:
# Replace target value 4 to 1
df.replace({'target':{4:1}}, inplace=True)
df['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

- 0 --> negative tweet
- 1 --> positive tweet

**Stemming**

Stemming is the process of reducing a word to its Root word
- ex. actor, actress, acting = act

In [22]:
port_stem = PorterStemmer()

In [23]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ', content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)

  return stemmed_content

In [24]:
df['stemmed_content'] = df['text'].apply(stemming) # 50 minutes to complete this execution

In [25]:
df.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [26]:
print(df['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [27]:
print(df['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [28]:
X = df['stemmed_content'].values
y = df['target'].values

In [29]:
print(X)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [30]:
print(y)

[0 0 0 ... 1 1 1]


Splitting data for train/test

In [33]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=1)

In [34]:
# Check shapes of the split data
print(X.shape, x_train.shape, x_test.shape)
print(y.shape, y_train.shape, y_test.shape)

(1600000,) (1280000,) (320000,)
(1600000,) (1280000,) (320000,)


In [35]:
# Converting string data to numerical data
vectorizer = TfidfVectorizer()

x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)

In [36]:
print(x_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 9455726 stored elements and shape (1280000, 461471)>
  Coords	Values
  (0, 341505)	0.41911697481409166
  (0, 364818)	0.37627918401650134
  (0, 239610)	0.20930055750051507
  (0, 266342)	0.3358584555454906
  (0, 227941)	0.2912488589003264
  (0, 317916)	0.30350077343669557
  (0, 347931)	0.4123017384299885
  (0, 106816)	0.37134449965091226
  (0, 408994)	0.20327132937613104
  (1, 286004)	0.4978274292193412
  (1, 181329)	0.5247713457687214
  (1, 411415)	0.2891245460020964
  (1, 240357)	0.3275728915457943
  (1, 324566)	0.5346829742944464
  (2, 118467)	0.8675495656028976
  (2, 129982)	0.27792888797524024
  (2, 306993)	0.41244791725873703
  (3, 411415)	0.1358344986846516
  (3, 170998)	0.13452947475642427
  (3, 420756)	0.17426809132378654
  (3, 78568)	0.1444559981151369
  (3, 357356)	0.468559153733546
  (3, 456847)	0.24416650707154486
  (3, 300444)	0.19284721323122722
  (3, 375838)	0.24345453849281473
  :	:
  (1279995, 411671)	0.37283

In [37]:
print(x_test)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2286456 stored elements and shape (320000, 461471)>
  Coords	Values
  (0, 42924)	0.32014153948273183
  (0, 124254)	0.3248820736879638
  (0, 229447)	0.4561274766251285
  (0, 234503)	0.24661051701774586
  (0, 237071)	0.3431859054639528
  (0, 408994)	0.26064591774569545
  (0, 445677)	0.48810739890224003
  (0, 446185)	0.3148496750005442
  (1, 67632)	0.3558726474087863
  (1, 136424)	0.3159549044093268
  (1, 212954)	0.2815630868489278
  (1, 240137)	0.2747376774458424
  (1, 240357)	0.23336407832693098
  (1, 283967)	0.5423261438034869
  (1, 368868)	0.30322358217997436
  (1, 408994)	0.2086630360521934
  (1, 414061)	0.3670250007983236
  (2, 7668)	0.44871661847697
  (2, 142305)	0.21104290585609561
  (2, 145591)	0.2900799045239692
  (2, 151588)	0.1528012728054361
  (2, 181201)	0.30987111374373477
  (2, 242883)	0.36935042060735335
  (2, 285703)	0.32899661981583517
  (2, 285719)	0.48791097125844635
  :	:
  (319996, 437575)	0.6505512796264

Training ML model: Logistic Regression

In [38]:
model = LogisticRegression(max_iter=1000)

model.fit(x_train, y_train)

LogisticRegression(max_iter=1000)

Model Evaluation

In [39]:
prediction = model.predict(x_train)
training_data_accuracy = accuracy_score(y_train, prediction)
print('Accuracy score on train: ', training_data_accuracy)

Accuracy score on train:  0.80221796875


In [40]:
test_prediction = model.predict(x_test)
test_data_accuracy = accuracy_score = accuracy_score(y_test, test_prediction)
print('Accuracy score on test:', test_data_accuracy)

Accuracy score on test: 0.77814375


Saving trained model

In [41]:
import pickle

In [43]:
filename = 'trained_model.sav'
pickle.dump(model, open(filename, 'wb'))

Using saved model for future predictions

In [ ]:
# loading saved model
loaded_model = pickle.load(open('trained_model.sav', 'rb'))

In [51]:
X_new = x_test[100]
print(y_test[100])

prediction = model.predict(X_new)
if (prediction[0] == 0):
  print('Negative tweet')
else:
  print('Positive tweet')

0
Negative tweet
